# Data Preparation

In [1]:
# fld_list = [
# 'B2P2EC2PN2PNTxt2PNSctn2PNSctnTxt2PNSent2PNSubTk2PNSubTkCharGrn',
# 'B2P2EC2PN2PNTxt2PNSctn2PNSctnTxt2PNSent2PNSubTk2PNSubTkWoodRootGrn',
    
# 'B2P2EC2PN2PNTxt2PNSctn2PNSctnNm2PNSctnNmTkGrn', 
# 'B2P2EC2PN2PNTxt2PNSctn2PNSctnTxt2PNSent2PNTkGrn',
# 'B2P2EC2PN2PNTxt2PNSctn2PNSctnTxt2PNSent2PNPOSGrn',

# 'B2P2EC2PN2PNDT2PNDTdftGrn',
    
# 'B2P2EC2Diag2DiagV2DiagVdftGrn',
# 'B2P2EC2Diag2DiagDT2DiagDTdftGrn',
# 'B2P2EC2Med2MedV2MedVdftGrn',
# 'B2P2EC2Med2MedDT2MedDTdftGrn',]

fld_list = [
'B2P2EC2Diag2DiagV2DiagVdftGrn',
'B2P2EC2Diag2DiagDT2DiagDTdftGrn',
'B2P2EC2Med2MedV2MedVdftGrn',
'B2P2EC2Med2MedDT2MedDTdftGrn',]

In [2]:
import numpy as np
import pandas as pd
df = pd.read_pickle('df_batch_np.p')
df['max'] = df['value'].apply(lambda x: np.array(x).max())#.shift()
df['shape'] = df['value'].apply(lambda x: np.array(x).shape)#.shift()
df

,layer,name,value,max,shape
0,0,B_lnP,3,3.0,()
1,1,B2P_lnEC,"[6, 5, 3]",6.0,"(3,)"
2,2,B2P2EC_lnDiag,"[[1, 3, 1, 4, 2, 3], [4, 1, 3, 2, 2, 0], [3, 1...",4.0,"(3, 6)"
3,3,B2P2EC2Diag_lnDiagV,"[[[1, 0, 0, 0], [1, 1, 1, 0], [1, 0, 0, 0], [1...",1.0,"(3, 6, 4)"
4,4,B2P2EC2Diag2DiagV_lnDiagVdftGrn,"[[[[2], [0], [0], [0]], [[2], [2], [2], [0]], ...",2.0,"(3, 6, 4, 1)"
5,5,B2P2EC2Diag2DiagV2DiagVdftGrn_idx,"[[[[[2. 3.]], [[0. 0.]], [[0. 0.]], [[0. 0.]]]...",15.0,"(3, 6, 4, 1, 2)"


In [3]:
info_idx = df['value'].values[-1]
info_idx.shape

(3, 6, 4, 1, 2)

In [4]:
[len(i.split('2')) - 1 for i in fld_list]

[5, 5, 5, 5]

In [5]:
fld2info_idx_dict = {}
for fld in fld_list:
    fld2info_idx_dict[fld] = info_idx
    
for k, v in fld2info_idx_dict.items(): print(k, v.shape)

B2P2EC2Diag2DiagV2DiagVdftGrn (3, 6, 4, 1, 2)
B2P2EC2Diag2DiagDT2DiagDTdftGrn (3, 6, 4, 1, 2)
B2P2EC2Med2MedV2MedVdftGrn (3, 6, 4, 1, 2)
B2P2EC2Med2MedDT2MedDTdftGrn (3, 6, 4, 1, 2)


# Model Structure Info

In [6]:
def get_dataflow_info(fld_list):

    df = pd.DataFrame([i.split('2') for i in fld_list])

    L = []
    for idx, row in df.iterrows():
        for layer_idx in range(1, len(row) - 1):

            # f'{layer_idx}-{(layer_idx + 1)}' + ':' +
            a, b = layer_idx, layer_idx + 1
            if row[b] == None: continue
            if row[a] == None: continue
            layer_nn_name =  f'{a}-{b}:{row[a]}-{row[b]}'
            if layer_nn_name not in L:
                L.append(layer_nn_name)
    # print(L)
    layers = [i.split(':')[0] for i in L]
    # print(layers)
    info = pd.DataFrame({'layers': layers, 'nn': L}).sort_values('nn').reset_index(drop = True)
    return info


def get_merge_layernn(x):
    D = {}
    layer_name = x[0].split(':')[0]
    for i in x:
        i = i.split(':')[-1]
        a, b = i.split('-')
        if a not in D:
            D[a] = [b]
        else:
            D[a].append(b)
    
    L = []
    for parent, childrens in D.items():
        if len(childrens) >= 2:
            L.append(layer_name + ':' + parent + '-' + '&'.join(childrens))
    return L


def get_single_layernn(x):
    D = {}
    layer_name = x[0].split(':')[0]
    for i in x:
        i = i.split(':')[-1]
        a, b = i.split('-')
        if a not in D:
            D[a] = [b]
        else:
            D[a].append(b)
    
    L = []
    for parent, childrens in D.items():
        if len(childrens) == 1:
            L.append(layer_name + ':' + parent + '-' + '&'.join(childrens))
    return L
    

In [7]:
def generate_df_struct(fld_list, mergefirst_fld_list):
    info = get_dataflow_info(fld_list)
    df_struct = info.groupby('layers').apply(lambda x: x['nn'].to_list()).reset_index()
    df_struct.columns = ['layers', 'nn']
    # df_struct['grn'] = df_struct['nn'].apply(lambda x: [i for i in x if 'Grn' in i])
    # df_struct['fld'] = df_struct['nn'].apply(lambda x: [i for i in x if 'Grn' not in i])
    # df_struct

    df_struct['single'] = df_struct['nn'].apply(lambda x: get_single_layernn(x))
    df_struct['merge'] = df_struct['nn'].apply(lambda x: get_merge_layernn(x))

    df_struct['mergefirst'] = df_struct['merge'].apply(lambda x: [i for i in x if i.split('-')[-1] in mergefirst_fld_list])
    df_struct['mergelast'] = df_struct['merge'].apply(lambda x: [i for i in x if i.split('-')[-1] not in mergefirst_fld_list])
    df_struct = df_struct.drop(columns = ['merge'])
    return df_struct



In [8]:
print(len(fld2info_idx_dict))
for k, v in fld2info_idx_dict.items(): print(k, v.shape)
fld_list = [i for i in fld2info_idx_dict]
print(fld_list)

4
B2P2EC2Diag2DiagV2DiagVdftGrn (3, 6, 4, 1, 2)
B2P2EC2Diag2DiagDT2DiagDTdftGrn (3, 6, 4, 1, 2)
B2P2EC2Med2MedV2MedVdftGrn (3, 6, 4, 1, 2)
B2P2EC2Med2MedDT2MedDTdftGrn (3, 6, 4, 1, 2)
['B2P2EC2Diag2DiagV2DiagVdftGrn', 'B2P2EC2Diag2DiagDT2DiagDTdftGrn', 'B2P2EC2Med2MedV2MedVdftGrn', 'B2P2EC2Med2MedDT2MedDTdftGrn']


In [9]:
mergefirst_fld_list = [
    'DiagDT&DiagV',
    'MedDT&MedV',
    'PNDT&PNTxt',
    'PNSctnNm&PNSctnTxt',
    'PNPOSGrn&PNSubTk&PNTkGrn', # TODO: notice the order
]

mergefirst_fld_list = ['&'.join(sorted(i.split('&'))) for i in mergefirst_fld_list]
df_struct = generate_df_struct(fld_list, mergefirst_fld_list)
df_struct

,layers,nn,single,mergefirst,mergelast
0,1-2,[1-2:P-EC],[1-2:P-EC],[],[]
1,2-3,"[2-3:EC-Diag, 2-3:EC-Med]",[],[],[2-3:EC-Diag&Med]
2,3-4,"[3-4:Diag-DiagDT, 3-4:Diag-DiagV, 3-4:Med-MedD...",[],"[3-4:Diag-DiagDT&DiagV, 3-4:Med-MedDT&MedV]",[]
3,4-5,"[4-5:DiagDT-DiagDTdftGrn, 4-5:DiagV-DiagVdftGr...","[4-5:DiagDT-DiagDTdftGrn, 4-5:DiagV-DiagVdftGr...",[],[]


In [10]:
LayerTrans2LayerNmae = {}
for idx, d in df_struct.iloc[::-1].iterrows():
    name = d['layers']
    single_list = d['single']
    mergefirst_list = d['mergefirst']
    mergelast_list = d['mergelast']
    layer_list = single_list + mergefirst_list + mergelast_list
    # print(name, len(layer_list), layer_list)
    LayerTrans2LayerNmae[name] = layer_list
    
LayerTrans2LayerNmae

{'4-5': ['4-5:DiagDT-DiagDTdftGrn',
  '4-5:DiagV-DiagVdftGrn',
  '4-5:MedDT-MedDTdftGrn',
  '4-5:MedV-MedVdftGrn'],
 '3-4': ['3-4:Diag-DiagDT&DiagV', '3-4:Med-MedDT&MedV'],
 '2-3': ['2-3:EC-Diag&Med'],
 '1-2': ['1-2:P-EC']}

# Data Flow

In [11]:
len(fld2info_idx_dict)

4

In [12]:
name2tensor_dict = {}
for fld, tensor in fld2info_idx_dict.items():
    name = fld.split('2')[-1]
    name2tensor_dict[name] = tensor

for k, v in name2tensor_dict.items(): print(k, v.shape)

DiagVdftGrn (3, 6, 4, 1, 2)
DiagDTdftGrn (3, 6, 4, 1, 2)
MedVdftGrn (3, 6, 4, 1, 2)
MedDTdftGrn (3, 6, 4, 1, 2)


In [13]:

for layername in LayerTrans2LayerNmae['4-5']:
    # print()
    print('\n')
    output, inputs = layername.split(':')[-1].split('-')
    inputs = inputs.split('&')
    print(layername)
    print(output,'<-----',  inputs)
    print('input info:')
    
    # From inputs to output
    for input_name in inputs:
        input_tensor = name2tensor_dict[input_name]
        print(input_name, input_tensor.shape)
        
        if 'Grn' in input_name:
            input_tensor = np.stack([input_tensor]*10, axis=-1)
            print('    ---> Grain idx, Expander Dims', input_tensor.shape)
    
    output_tensor = input_tensor.mean(-2)
    print(output, output_tensor.shape)
    
    name2tensor_dict[output] = output_tensor
    



4-5:DiagDT-DiagDTdftGrn
DiagDT <----- ['DiagDTdftGrn']
input info:
DiagDTdftGrn (3, 6, 4, 1, 2)
    ---> Grain idx, Expander Dims (3, 6, 4, 1, 2, 10)
DiagDT (3, 6, 4, 1, 10)


4-5:DiagV-DiagVdftGrn
DiagV <----- ['DiagVdftGrn']
input info:
DiagVdftGrn (3, 6, 4, 1, 2)
    ---> Grain idx, Expander Dims (3, 6, 4, 1, 2, 10)
DiagV (3, 6, 4, 1, 10)


4-5:MedDT-MedDTdftGrn
MedDT <----- ['MedDTdftGrn']
input info:
MedDTdftGrn (3, 6, 4, 1, 2)
    ---> Grain idx, Expander Dims (3, 6, 4, 1, 2, 10)
MedDT (3, 6, 4, 1, 10)


4-5:MedV-MedVdftGrn
MedV <----- ['MedVdftGrn']
input info:
MedVdftGrn (3, 6, 4, 1, 2)
    ---> Grain idx, Expander Dims (3, 6, 4, 1, 2, 10)
MedV (3, 6, 4, 1, 10)


In [14]:

for layername in LayerTrans2LayerNmae['3-4']:
    # print()
    print('\n')
    output, inputs = layername.split(':')[-1].split('-')
    inputs = inputs.split('&')
    print(layername)
    print(output,'<-----',  inputs)
    print('input info:')
    
    
    # From inputs to output
    for input_name in inputs:
        input_tensor = name2tensor_dict[input_name]
        print(input_name, input_tensor.shape)
        
        if 'Grn' in input_name:
            input_tensor = np.stack([input_tensor]*10, axis=-1)
            print('---> Grain idx, Expander Dims', input_tensor.shape)
    
    output_tensor = input_tensor.mean(-2)
    print(output, output_tensor.shape)
    
    name2tensor_dict[output] = output_tensor
    



3-4:Diag-DiagDT&DiagV
Diag <----- ['DiagDT', 'DiagV']
input info:
DiagDT (3, 6, 4, 1, 10)
DiagV (3, 6, 4, 1, 10)
Diag (3, 6, 4, 10)


3-4:Med-MedDT&MedV
Med <----- ['MedDT', 'MedV']
input info:
MedDT (3, 6, 4, 1, 10)
MedV (3, 6, 4, 1, 10)
Med (3, 6, 4, 10)


In [15]:

for layername in LayerTrans2LayerNmae['2-3']:
    # print()
    print('\n')
    output, inputs = layername.split(':')[-1].split('-')
    inputs = inputs.split('&')
    print(layername)
    print(output,'<-----',  inputs)
    print('input info:')
    
    
    # From inputs to output
    for input_name in inputs:
        input_tensor = name2tensor_dict[input_name]
        print(input_name, input_tensor.shape)
        
        if 'Grn' in input_name:
            input_tensor = np.stack([input_tensor]*10, axis=-1)
            print('---> Grain idx, Expander Dims', input_tensor.shape)
    
    output_tensor = input_tensor.mean(-2)
    print(output, output_tensor.shape)
    
    name2tensor_dict[output] = output_tensor
    



2-3:EC-Diag&Med
EC <----- ['Diag', 'Med']
input info:
Diag (3, 6, 4, 10)
Med (3, 6, 4, 10)
EC (3, 6, 10)


In [16]:

for layername in LayerTrans2LayerNmae['1-2']:
    # print()
    print('\n')
    output, inputs = layername.split(':')[-1].split('-')
    inputs = inputs.split('&')
    print(layername)
    print(output,'<-----',  inputs)
    print('input info:')
    
    
    # From inputs to output
    for input_name in inputs:
        input_tensor = name2tensor_dict[input_name]
        print(input_name, input_tensor.shape)
        
        if 'Grn' in input_name:
            input_tensor = np.stack([input_tensor]*10, axis=-1)
            print('---> Grain idx, Expander Dims', input_tensor.shape)
    
    output_tensor = input_tensor.mean(-2)
    print(output, output_tensor.shape)
    
    name2tensor_dict[output] = output_tensor
    



1-2:P-EC
P <----- ['EC']
input info:
EC (3, 6, 10)
P (3, 10)


# EMR-L

In [85]:
'3-4:Diag-DiagDT&DiagV'

'3-4:Diag-DiagDT&DiagV'